In [ ]:
import pandas as pd
from os import walk

In [ ]:
datasets_names = []
path_df = '../datasets/original_files/'
for (dirpath, dirnames, filenames) in walk(path_df):
    datasets_names.extend(filenames)
    break
datasets_names

In [ ]:
dataFrame = False
first = True
for name in datasets_names:
    if(first):
        first = False
        dataFrame = pd.read_csv(path_df+name, compression='gzip')
        dataFrame.set_index('id', drop=False, inplace=True)
        print('adding', len(dataFrame), 'rows')
    else:
        _tmpDf = pd.read_csv(path_df+name, compression='gzip')
        _tmpDf.set_index('id', drop=False, inplace=True)
        print('adding', len(_tmpDf), 'rows')
        dataFrame = pd.concat([dataFrame, _tmpDf])
        del _tmpDf
print('total rows:', len(dataFrame))
dataFrame.head()

In [ ]:
#cleaning columns
for x in dataFrame.columns:
    print(x)
    values = dataFrame[x][pd.notna(dataFrame[x])].values
    if(len(values) > 0):
        print(values[0])
    else:
        print('NOT_VALUES')
    print('')

In [ ]:
# neighbourhood
# neighbourhood_cleansed
# neighbourhood_group_cleansed
# guests_included
# license
# is_business_travel_ready

cols_to_drop = [
    'market', 'street',
    'listing_url', 'scrape_id', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url',
    'host_name', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
    'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode', 'smart_location', 'country_code',
    'weekly_price', 'monthly_price', 'security_deposit', 'cleaning_fee', 'extra_people',
    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights',
    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
    'requires_license','jurisdiction_names','host_location'
]
# [colName for colName in dataFrame.columns if colName not in cols_to_drop]


dataFrame.drop(cols_to_drop, axis=1, inplace=True)

In [ ]:
#text pipeline steps
from sklearn.base import BaseEstimator, TransformerMixin

#Esta clase simplemente filtra las columnas que se le indica en el constructor
class FeatureSelector( BaseEstimator, TransformerMixin ):
    #Class Constructor 
    def __init__( self, feature_names ):
        self._feature_names = feature_names 
    
    #Return self nothing else to do here    
    def fit( self, X, y = None ):
        return self 
    
    #Method that describes what we need this transformer to do
    def transform( self, X, y = None ):
        return X[ self._feature_names ]
    
class TextTransformer(BaseEstimator, TransformerMixin):
    
    def __clean_text(self, x):
        for punct in "/-'":
            x = x.replace(punct, ' ')
        for punct in '&':
            x = x.replace(punct, f' {punct} ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~•' + '“”’':
            x = x.replace(punct, '')
        return x.lower()
    
    def __parseTextCols(self, x):
        finalTexts = []
        for i in x:
            if(pd.notna(i) and i not in finalTexts):
                finalTexts.append(i)
        text = self.__clean_text(' '.join(finalTexts))
        return text
    
    def fit(self, X, y = None):
        return self
    
    def transform (self, X, y = None):
        return X.apply(self.__parseTextCols, axis=1)
    
    
from sklearn.feature_extraction.text import TfidfVectorizer
class custom_Tfidf(TfidfVectorizer, TransformerMixin):
    options= {
        'fitSample': 1
    }
    def __init__(self, params, options = None):
        self.vectorizer = TfidfVectorizer(**params)
        if(options != None):
            for key in options.keys():
                self.options[key] = options[key]
        
    def fit(self, X, y = None):
        self.vectorizer.fit(X.sample(frac=self.options['fitSample']))
        return self
    
    def transform(self, X, y = None):
        print(len(X))
        return self.vectorizer.transform(X)

In [ ]:
#pipeline
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

text_cols = ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"]

from nltk.corpus import stopwords
stopwords_en = stopwords.words('english')
textVectSettings = {
    'stop_words': stopwords_en,
    'max_df': 0.90,
    'min_df': .05,
    'ngram_range': (1,3),
    'max_features': 350
}

#Pasos para el pipeline Textos
text_pipeline = Pipeline(steps = [
    ( 'text_selector', FeatureSelector(text_cols) ),
    ( 'text_transformer', TextTransformer() ),
    ( 'text_vectorize',  custom_Tfidf(textVectSettings, {'fitSample':.01}))
] )

text_pipeline.fit(dataFrame)
# text_pipeline.transform(dataFrame)

In [ ]:
dummy_cols = ["instant_bookable","is_business_travel_ready","cancellation_policy",
"require_guest_phone_verification",
"require_guest_profile_picture","host_response_time",
"host_is_superhost","host_has_profile_pic","host_identity_verified",
"city","state","property_type","room_type","bed_type"];

dummy_pipeline = Pipeline(steps = [
    ('dummy_selector', FeatureSelector(dummy_cols))
])

In [ ]:
# import nltk
# nltk.download('stopwords')

class customTransformer:
    
    textCols = []
    configuration = {
        'textTrainSample': .3
    }
    
    def __init__(self, columns, configuration = False):
        self.fitted = False
        from nltk.corpus import stopwords
        from sklearn.feature_extraction.text import TfidfTransformer
        from sklearn.feature_extraction.text import TfidfVectorizer
        stopwords_en = stopwords.words('english')
        
        textVectSettings = {
            'stop_words': stopwords_en,
            'max_df': 0.90,
            'min_df': .05,
            'ngram_range': (1,3),
            'max_features': 350
        }
        
        if(configuration != False):
            for key in configuration.keys():
                self.configuration[key] = configuration[key]
        self.Tfid = TfidfVectorizer(**textVectSettings)
        self.textCols = columns['textCols']
        
    def __clean_text(self, x):
        for punct in "/-'":
            x = x.replace(punct, ' ')
        for punct in '&':
            x = x.replace(punct, f' {punct} ')
        for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~•' + '“”’':
            x = x.replace(punct, '')
        return x.lower()
    
    def __parseTextCols(self, x):
        finalTexts = []
        for x in x[self.textCols]:
            if(pd.notna(x) and x not in finalTexts):
                finalTexts.append(x)
        text = self.__clean_text(' '.join(finalTexts))
        return text
    
    def fit_transform(self, x):
        self.fit(x)
        return self.transform(x)
    
    def fit(self, x):
        """
            Learn parameters from pandas dataframe.
        """
        sample = self.configuration['textTrainSample']
        texts = x.sample(frac=sample).apply(self.__parseTextCols, axis=1)
        self.Tfid.fit(texts)
        self.fitted= True
        
    def transform(self, x):
        """
            Transform pandas dataframe to matrix.
            ----
            Retruns matrix.
        """
        from scipy.sparse import hstack
        if(not self.fitted):
            raise NameError('customTransformer not fitted')
        texts = x.apply(self.__parseTextCols, axis=1)
        textVectors = self.Tfid.transform(texts)
        return hstack([textVectors])
    
    def save_pkl(slef, filename):
        """
            Save class with parameters to a .pkl file.
        """
        with open(filename, 'wb') as output:
            pickle.dump(self, output, pickle.HIGHEST_PROTOCOL)
    
        
cols = {
    'textCols': ["name","summary","space","description","neighborhood_overview","notes","transit","access",
             "interaction","house_rules","host_about"]
}
configuration = {
    'textTrainSample': .01
}
transformer = customTransformer(cols, configuration)
transformer.fit(dataFrame)